# Notebook 1 — Frames & Homogeneous Transforms (SE(3) Pose Algebra)


**Aim:** Build rock-solid intuition for frames and transform chaining so you stop losing time to “wrong order” bugs.

**You will be able to:**
- Interpret a transform \(T^A_B\) as “coordinates in B → coordinates in A”
- Compose transforms with correct order, invert transforms, and validate them
- Debug frame bugs using the “inner-frame-matching” rule


In [ ]:
# Common setup for all notebooks
import numpy as np
import math
import matplotlib.pyplot as plt

# For 3D plots
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

np.set_printoptions(precision=4, suppress=True)


In [ ]:
# --- Minimal "Modern Robotics style" utilities (self-contained) ---

def skew(w):
    'Return [w]x (3x3 skew-symmetric) for w = (wx,wy,wz).'
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]], dtype=float)

def so3_exp(omega, theta):
    'Rodrigues: exp([omega] theta). omega can be non-unit; theta in radians.'
    omega = np.asarray(omega, dtype=float).reshape(3)
    norm = np.linalg.norm(omega)
    if norm < 1e-12:
        return np.eye(3)
    w = omega / norm
    th = theta * norm
    W = skew(w)
    return np.eye(3) + math.sin(th)*W + (1-math.cos(th))*(W@W)

def so3_log(R):
    'Return (omega_hat, theta) such that R = exp([omega_hat] theta), omega_hat unit.'
    R = np.asarray(R, dtype=float).reshape(3,3)
    tr = np.trace(R)
    cos_th = (tr - 1)/2
    cos_th = np.clip(cos_th, -1.0, 1.0)
    theta = math.acos(cos_th)
    if abs(theta) < 1e-12:
        return np.array([1.0, 0.0, 0.0]), 0.0
    if abs(theta - math.pi) < 1e-6:
        # Near pi: robust axis extraction from diagonal
        v = np.sqrt(np.maximum((np.diag(R)+1)/2, 0))
        # Fix signs using off-diagonals
        if R[2,1] - R[1,2] < 0: v[0] = -v[0]
        if R[0,2] - R[2,0] < 0: v[1] = -v[1]
        if R[1,0] - R[0,1] < 0: v[2] = -v[2]
        omega_hat = v / (np.linalg.norm(v) + 1e-12)
        return omega_hat, theta
    W = (R - R.T) / (2*math.sin(theta))
    omega_hat = np.array([W[2,1], W[0,2], W[1,0]])
    omega_hat /= (np.linalg.norm(omega_hat)+1e-12)
    return omega_hat, theta

def make_T(R, p):
    'Homogeneous transform from rotation R (3x3) and position p (3,).'
    T = np.eye(4)
    T[:3,:3] = np.asarray(R, dtype=float).reshape(3,3)
    T[:3,3]  = np.asarray(p, dtype=float).reshape(3)
    return T

def inv_T(T):
    'Inverse of SE(3) transform.'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    Rt = R.T
    return make_T(Rt, -Rt@p)

def adjoint(T):
    'Adjoint matrix Ad_T (6x6) mapping twists between frames.'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    P = skew(p)
    Ad = np.zeros((6,6))
    Ad[:3,:3] = R
    Ad[3:,3:] = R
    Ad[3:,:3] = P@R
    return Ad

def se3_exp(S, theta):
    'SE(3) exponential: exp([S] theta), where S is 6-vector screw axis.'
    S = np.asarray(S, dtype=float).reshape(6)
    w = S[:3]
    v = S[3:]
    normw = np.linalg.norm(w)
    if normw < 1e-12:
        R = np.eye(3)
        p = v * theta
        return make_T(R, p)
    w_hat = w / normw
    th = theta * normw
    R = so3_exp(w_hat, th)
    W = skew(w_hat)
    G = (np.eye(3)*th + (1-math.cos(th))*W + (th - math.sin(th))*(W@W))
    p = (G @ (v / normw))
    return make_T(R, p)

def se3_log(T):
    'SE(3) log: return V (6,) and theta such that T = exp([V] theta).'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    w_hat, th = so3_log(R)
    if abs(th) < 1e-12:
        V = np.concatenate([np.zeros(3), p])
        return V, 1.0
    W = skew(w_hat)
    A = (np.eye(3) - 0.5*W + (1/th - 0.5/math.tan(th/2))*(W@W))
    v = A @ p
    V = np.concatenate([w_hat, v])
    return V, th

def FK_space(Slist, thetalist, M):
    'PoE forward kinematics in space form.'
    T = np.eye(4)
    for S, th in zip(Slist.T, thetalist):
        T = T @ se3_exp(S, th)
    return T @ M

def Jacobian_space(Slist, thetalist):
    'Space Jacobian J_s(theta). Slist: 6xn. thetalist: n.'
    n = Slist.shape[1]
    J = np.zeros((6,n))
    T = np.eye(4)
    for i in range(n):
        if i == 0:
            J[:,0] = Slist[:,0]
        else:
            T = T @ se3_exp(Slist[:,i-1], thetalist[i-1])
            J[:,i] = adjoint(T) @ Slist[:,i]
    return J

def damped_pinv(J, lam=1e-3):
    'Damped least squares pseudo-inverse.'
    m, n = J.shape
    return J.T @ np.linalg.inv(J@J.T + (lam**2)*np.eye(m))

# --- plotting helpers ---
def plot_frame(ax, T=np.eye(4), name=None, length=0.08):
    'Plot a coordinate frame (3 axes).'
    T = np.asarray(T, float)
    o = T[:3,3]
    R = T[:3,:3]
    xs = o + R[:,0]*length
    ys = o + R[:,1]*length
    zs = o + R[:,2]*length
    ax.plot([o[0], xs[0]],[o[1], xs[1]],[o[2], xs[2]], linewidth=2)
    ax.plot([o[0], ys[0]],[o[1], ys[1]],[o[2], ys[2]], linewidth=2)
    ax.plot([o[0], zs[0]],[o[1], zs[1]],[o[2], zs[2]], linewidth=2)
    if name:
        ax.text(o[0], o[1], o[2], f' {name}')

def set_axes_equal(ax, lim=0.4):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(0, 2*lim)
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')


## 1) The one rule: \(p_A = T^A_B\,p_B\)

- \(T^A_B\): converts coordinates **from frame B into frame A**.
- The *numbers* of the same geometric point change depending on the frame.


In [ ]:
p_B = np.array([0.2, 0.0, 0.0, 1.0])

R_AB = so3_exp([0,0,1], math.radians(45))
p_AB = np.array([0.1, 0.05, 0.0])
T_AB = make_T(R_AB, p_AB)

p_A = T_AB @ p_B
print("T_AB=\n", T_AB)
print("p_A=", p_A)


## 2) LEGO intuition: chain transforms like snapping bricks

If you know \(T^A_B\) and \(T^B_C\), then:

\[
T^A_C = T^A_B\,T^B_C
\]

**Debug trick:** Multiply only when the **inner frames match**:
\[
(A\leftarrow B)(B\leftarrow C) = (A\leftarrow C)
\]


In [ ]:
# Example chain: tag -> camera -> world -> robot

T_ct = make_T(so3_exp([0,1,0], math.radians(20)), [0.30, 0.05, 0.60])  # t -> c
T_wc = make_T(so3_exp([0,0,1], math.radians(-30)), [1.20, -0.40, 1.50]) # c -> w
T_wr = make_T(so3_exp([0,0,1], math.radians(90)), [1.00, 0.20, 0.00])  # r -> w

T_rw = inv_T(T_wr)
T_rt = T_rw @ T_wc @ T_ct  # t -> r

print("T_rt=\n", T_rt)


## 3) Visualize the frames to see composition


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

plot_frame(ax, np.eye(4), "w")
plot_frame(ax, T_wc, "c")
plot_frame(ax, T_wc @ T_ct, "t (in w)")
plot_frame(ax, T_wr, "r (in w)")

set_axes_equal(ax, lim=1.2)
plt.title("Frames in World Coordinates (w)")
plt.show()


## 4) Mini self-quiz

1) If you have \(T^w_c\) and \(T^c_t\), what is \(T^w_t\)?  
2) If a library returns “pose of tag in camera”, is that \(T^c_t\) or \(T^t_c\)?  
3) What’s the quickest sanity check after chaining (e.g., invert then re-compose)?
